In [30]:
%config Completer.use_jedi = False
import matplotlib
matplotlib.use('TkAgg')


In [31]:
import os
import numpy as np
import pandas as pd
print(pd.__version__)
from operator import truediv, mul
import array
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib import lines


1.4.4


In [32]:
import seaborn as sns

In [33]:
def resampling(map_data, long=False):
    v = []
    area = []
    ent = []
    cnt = 0
    for i in map_data.item()['time']:
        v.append(float(i.to_sec()))
        if(map_data.item()['area'][cnt] != 0):
            area.append(map_data.item()['area'][cnt])
        else:
            area.append(0.0025)
        ent.append(map_data.item()['total_entropy'][cnt])
        cnt += 1
    d = pd.DataFrame(columns=['ts','area','ent'])

    d.loc[:,'ts'] = v
    d.loc[:,'area'] = area
    d.loc[:,'ent'] = ent
    d = d.set_index('ts')
    d.index = pd.to_datetime(d.index, unit='s')
    
    tmp = d.resample('2S')[['area']].agg(lambda x : np.nan if x.count() == 0 else x.idxmax()).ffill()

    df = d.loc[tmp['area']]
    cnt = 0
    if (not long):
        while (len(df) < 300):
            new_row = pd.DataFrame(columns=['area','ent'])
            new_row.loc[0 + cnt / 1000.0,'area'] = 0.0025
            new_row.loc[0 + cnt / 1000.0,'ent'] = 1
            cnt += 1
            df = pd.concat([new_row, df])
    else:
        while (len(df) < 450):
            new_row = pd.DataFrame(columns=['area','ent'])
            new_row.loc[0 + cnt / 1000.0,'area'] = 0.0025
            new_row.loc[0 + cnt / 1000.0,'ent'] = 1
            cnt += 1
            df = pd.concat([new_row, df])
        
    return df

In [34]:
#######################################################################################################################
#Author   : Dr. Arun B Ayyar
#
#Based on : Shimazaki H. and Shinomoto S., A method for selecting the bin size of a time histogram Neural Computation (2007)
#	   Vol. 19(6), 1503-1527
#
#Data     : The duration for eruptions of the Old Faithful geyser in Yellowstone National Park (in minutes) 
#	   or normal distribuition.
#	   given at http://176.32.89.45/~hideaki/res/histogram.html
#
#Comments : Implements a faster version than using hist from matplotlib and histogram from numpy libraries	
#           Also implements the shifts for the bin edges
#
########################################################################################################################


def compute_bin_size(data):
    data_max = max(data) #lower end of data
    data_min = min(data) #upper end of data
    n_min = 2500   #Minimum number of bins Ideal value = 2
    n_max = 2501  #Maximum number of bins  Ideal value =200
    n_shift = 1     #number of shifts Ideal value = 30
    N = np.array(range(n_min,n_max))
    D = float(data_max-data_min)/N    #Bin width vector
    Cs = np.zeros((len(D),n_shift)) #Cost function vector
    #Computation of the cost function
    for i in range(np.size(N)):
        shift = np.linspace(0,D[i],n_shift)
        for j in range(n_shift):
            edges = np.linspace(data_min+shift[j]-D[i]/2,data_max+shift[j]-D[i]/2,N[i]+1) # shift the Bin edges
            binindex = np.digitize(data,edges) #Find binindex of each data point
            ki= np.bincount(binindex)[1:N[i]+1] #Find number of points in each bin
            k = np.mean(ki) #Mean of event count
            v = sum((ki-k)**2)/N[i] #Variance of event count
            Cs[i,j]+= (2*k-v)/((D[i])**2) #The cost Function
    C=Cs.mean(1)

    #Optimal Bin Size Selection
    loc = np.argwhere(Cs==Cs.min())[0]
    cmin = C.min()
    idx  = np.where(C==cmin)
    idx = idx[0][0]
    optD = D[idx]
    return np.linspace(data_min+shift[loc[1]]-D[idx]/2,data_max+shift[loc[1]]-D[idx]/2,N[idx]+1,endpoint=False)

In [35]:
def get_rmse(occupancy, gt):
    oc = []
    se = 0
    for l in occupancy:
        p = float(l)
        oc.append(p)
    oc_gt = []
    for l in gt:
        p = float(l)
        oc_gt.append(p)
    for l in range(len(oc)):
        if(oc[l] == -1):
            oc[l] = 50
        if(oc_gt[l] == -1):
            oc_gt[l] = 50
        se += (oc[l] - oc_gt[l])**2
    mse = se/len(oc)
    return mse**0.5

def get_rmse_known(occupancy, gt):
    oc = []
    se = 0
    for l in occupancy:
        p = float(l)
        oc.append(p)
    oc_gt = []
    for l in gt:
        p = float(l)
        oc_gt.append(p)
    cnt = 0.0
    for l in range(len(oc)):
        if (oc[l] == -1):
            if (oc_gt[l] != -1):
                oc[l] = 50
                cnt += 1
                se += (oc[l] - oc_gt[l])**2
        else:
            if(oc_gt == -1):
                oc_gt[l] = 100 - oc[l]
            cnt += 1
            se += (oc[l] - oc_gt[l])**2
    mse = se/cnt
    return mse**0.5

def bac_manual(occupancy, gt):
    oc = []
    se = 0
    for l in occupancy:
        p = float(l)
        oc.append(p)
    oc_gt = []
    for l in gt:
        p = float(l)
        oc_gt.append(p)
    tot_free = 0
    tot_occ = 0
    corr_free = 0
    corr_occ = 0
    tot_unk = 0
    corr_unk = 0
    for l in range(len(oc)):
        if oc_gt[l] == 0:
            tot_free += 1
            if oc[l] == oc_gt[l]:
                corr_free += 1
        elif oc_gt[l] == 100:
            tot_occ += 1
            if oc[l] == oc_gt[l]:
                corr_occ += 1
        elif oc_gt[l] == -1:
            tot_unk += 1
            if oc[l] == oc_gt[l]:
                corr_unk += 1
    return [1/3.0*(corr_free/tot_free + corr_occ/tot_occ + corr_unk/tot_unk),corr_free/tot_free,corr_occ/tot_occ, corr_unk/tot_unk, 1/2.0*(corr_free/tot_free + corr_occ/tot_occ)]
    # return [1/2.0*(corr_free/tot_free + corr_occ/tot_occ),corr_free/tot_free,corr_occ/tot_occ, corr_unk/tot_unk]


In [36]:
def get_data(folder, columns, kind, df, low = 0, r = 10, ate = False, saving_results = False):
    color = ['tab:blue','tab:green','tab:red','tab:orange','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan','k']
    final_folders = []
    for k in kind:
        final_folders.append(os.path.join(folder, k))

    print(final_folders)

    cnt = 0
    
#     fig1, (ax1, ax2) = plt.subplots(1,2)    
    fig_map, (ax_map) = plt.subplots(1)
    fig_exp, (ax_exp) = plt.subplots(1)
#     fig_ent, (ax_ent) = plt.subplots(1)
#     fig_area, (ax_area) = plt.subplots(1)
    font = {
        'weight': 'normal',
        'size': 18,
            }
    ax_exp.set_ylabel('explored %', fontdict=font)
    ax_exp.set_xlabel('trajectory length [m]', fontdict=font)
    ax_map_split = ax_map.twinx()
    ax_map_split.set_ylabel('trajectory length [m]', fontdict=font)
    ax_map_split.set_ylim([0,120])

    sec_y = ax_exp.secondary_yaxis('right')
    sec_y.set_ylabel('normalized entropy', fontdict=font)
    sec_y.set_ylim([0,1])
    ax_map.set_ylabel('explored %', fontdict=font)
    ax_map.set_xlabel('time [s]', fontdict=font)
    ax_map.set_ylim([-0.050,1.050])
    
    for i in final_folders:
        tot_score = []
        tot_BAC =  []
        tot_BAC_free = []
        tot_BAC_occ = []
        if ate:
            tot_TRPE = []
            tot_RRPE = []
            tot_ATE = []
        cov_vec = []
        tot_lc = []
        glob_lc = []
        loc_lc = []
        tot_area = []
        area_bucketed = []
        entropy_bucketed = []
        pose_cov_vec = []
        final_map_rmse = []
        bac_manually = []
        bac_manually_unk = []
        bac_manually_free = []
        bac_manually_occ = []
        bac_manually_2 = []
        path_len =[]
        filtered_path_df = []
        wheel_tot = []
        for j in range(low,r):
            current = os.path.join(i,str(j))
            if (os.path.isfile(os.path.join(current,'map_data.npy'))):
                map_data = np.load(os.path.join(current,'map_data.npy'), allow_pickle=True, encoding="latin1")
                system_data = np.load(os.path.join(current,'feat.npy'), allow_pickle=True, encoding="latin1")
                wheel_data = np.load(os.path.join(current,'wheels.npy'), allow_pickle=True, encoding="latin1")
                general_results = open(os.path.join(current,'general_results.txt'), 'r')

                gt = open(os.path.join(folder,'gt_occupancy.txt'), 'r')
                occupancy = open(os.path.join(current,'occupancy.txt'), 'r')
                if("E1" in current):
                    occ = np.array(occupancy.read().splitlines())
                    occ = occ.reshape((621,-1))
                    ncelss = 2/0.05
                    occ_shift = np.roll(occ, (-int(ncelss),0), axis=(1,0))
                    new_occ = occ_shift.reshape(-1)
                    occupancy = map(int, new_occ)

                # occupancy_prob = open(os.path.join(current,'occupancy.txt'), 'r')
                occupancy_prob = occupancy
                # poses = open(os.path.join(current,'poses.g2o'), 'r')
                tot_lc.append(system_data.item()['total'][-1])
                glob_lc.append(system_data.item()['global'][-1])
                loc_lc.append(system_data.item()['local'][-1])
                tot_area.append(map_data.item()['area'][-1])
                wheel_tot.append(wheel_data.item()['tot_wheel'][-1])


                final_map_rmse.append(get_rmse(occupancy,gt))
                
                gt = open(os.path.join(folder,'gt_occupancy.txt'), 'r')
                occupancy = open(os.path.join(current,'occupancy.txt'), 'r')
                if("E1" in current):
                    occ = np.array(occupancy.read().splitlines())
                    occ = occ.reshape((621,-1))
                    ncelss = 2/0.05
                    occ_shift = np.roll(occ, (-int(ncelss),0), axis=(1,0))
                    new_occ = occ_shift.reshape(-1)
                    occupancy = map(int, new_occ)

                bac_m = bac_manual(occupancy,gt)
                bac_manually.append(bac_m[0])
                bac_manually_2.append(bac_m[-1])
                bac_manually_unk.append(bac_m[-2])
                bac_manually_free.append(bac_m[1])
                bac_manually_occ.append(bac_m[2])
                
                # binning the entropies and the area explored wrt 2-seconds time frames
                resampled = resampling(map_data, "_L" in current)
                if ("_L" in current):
                    area_bucketed.append(resampled[-450:].loc[:,'area'])            
                    entropy_sliced = resampled[-450:].loc[:,'ent']       
                else:
                    area_bucketed.append(resampled[-300:].loc[:,'area'])            
                    entropy_sliced = resampled[-300:].loc[:,'ent']       
                a = [0.0025 for i in range(len(area_bucketed[-1]))]
                res = list(map(truediv, area_bucketed[-1], a))
                res2 = list(map(truediv, entropy_sliced, res))
                print(res2[-1])
                entropy_bucketed.append(res2)

                # ATE - slam metric
                if ate:
                    path_gt = open(os.path.join(current,'rtabmap_odom.txt'),'r')
                    prev = [0,0,0]
                    first = True
                    path_df = pd.DataFrame(columns=['time','len'])
                    
                    for l in list(path_gt)[1:]:
                        p = l.split()
                        if first:
                            prev = [float(p[1]), float(p[2])]
                            path_df.loc[p[0],'time'] = float(p[0])
                            path_df.loc[p[0],'len'] = 0
                            cum_dist = 0
                            first = False
                        else:
                            cum_dist += ((prev[0]-float(p[1]))**2+(prev[1]-float(p[2]))**2)**0.5
                            
                            path_df.loc[p[0],'time'] = float(p[0])
                            path_df.loc[p[0],'len'] = cum_dist
                            
                            prev = [float(p[1]), float(p[2])]
                            
                    path_len.append(cum_dist)
                    path_df.index = pd.to_datetime(path_df.index, unit='s')
                    tmp_filtered_path_df = path_df.resample('2S').max().ffill()
                    if ("_L" not in current):
                        norm = 0
                        while (len(tmp_filtered_path_df) < 300):

                            new_row = pd.DataFrame(columns=['time','len'])
                            new_row.loc[0,'time'] = 0+ norm/1000.0
                            new_row.loc[0,'len'] = 0.00001
                            new_row = new_row.set_index('time')
                            new_row.index = pd.to_datetime(new_row.index, unit='s')
                            norm += 1
                            tmp_filtered_path_df = pd.concat([new_row, tmp_filtered_path_df])
                        filtered_path_df.append(tmp_filtered_path_df[-300:]['len'])
                    else:
                        norm = 0
                        while (len(tmp_filtered_path_df) < 450):

                            new_row = pd.DataFrame(columns=['time','len'])
                            new_row.loc[0,'time'] = 0 + norm/1000.0
                            norm += 1
                            new_row.loc[0,'len'] = 0.00001
                            new_row = new_row.set_index('time')
                            new_row.index = pd.to_datetime(new_row.index, unit='s')
                            tmp_filtered_path_df = pd.concat([new_row, tmp_filtered_path_df])
                            
                        filtered_path_df.append(tmp_filtered_path_df[-450:]['len'])

                    ate = open(os.path.join(current,'ate.txt'), 'r')
                    for l in ate:
                        if 'rmse' in l:
                            tot_ATE.append(float(l.split()[-2]))
                    
                    rpe = open(os.path.join(current,'rpe.txt'), 'r')
                    for l in rpe:
                        if 'translational' in l and 'rmse' in l:
                            tot_TRPE.append(float(l.split()[-2]))
                        elif 'rotational' in l and 'rmse' in l:
                            tot_RRPE.append(float(l.split()[-2]))
                            
                    odom = np.load(os.path.join(current,'odom.npy'), allow_pickle=True, encoding="latin1")
                    for k in range(len(odom.item()['time'])):
                        if (odom.item()['time'][-1].to_sec() - odom.item()['time'][k].to_sec() <= 603):
                            m = np.array(odom.item()['cov'][k]).reshape((6,6))
                            eigen = np.linalg.eig(m)
                            pose_cov_vec.append(np.exp(1/6.0 * np.sum(np.log(eigen[0]))).real)
                # collect map scores from general_results file
                for l in general_results:
                    if 'map_score' in l:
                        tot_score.append(float(l.split()[-1]))
                    if 'BAC score' in l:
                        tot_BAC.append(float(l.split()[-1]))
                    if 'correct_free_ratio' in l:
                        tot_BAC_free.append(float(l.split()[-1]))
                    if 'correct_occ_ratio' in l:
                        tot_BAC_occ.append(float(l.split()[-1]))

                # collect link variances of the graph
                # for l in poses:
                #     if 'EDGE' in l:
                #         s = l.split()
                #         m = np.array([[float(s[-6]), float(s[-5]), float(s[-4])],
                #                       [float(s[-5]), float(s[-3]), float(s[-2])],
                #                       [float(s[-4]), float(s[-2]), float(s[-1])]]
                #                     )
                #         cov = np.linalg.inv(m)
                #         eigen = np.linalg.eig(cov)
                #         cov_vec.append(np.exp(1/3.0 * np.sum(np.log(eigen[0]))).real)
                        #cov_vec.append(np.linalg.det(cov))
        if(len(tot_score)>0):
            wheel_over_path = list(map(truediv, wheel_tot, path_len))
            all_BAC.append(bac_manually)
            all_ATE.append(tot_ATE)
            all_ENT.append(entropy_bucketed)
            all_PtL.append(path_len)
            all_PmL.append(list(map(truediv, tot_lc, path_len)))

            if (saving_results):
                df.loc['BAC',i.split('/')[-1]] = 'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(bac_manually),3),round(np.std(bac_manually),3))
                df.loc['BAC_new',i.split('/')[-1]] = 'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(bac_manually_2),3),round(np.std(bac_manually_2),3))
                df.loc['per_m_wheel',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(wheel_over_path),3), + round(np.std(wheel_over_path),3))
            else:
                df.loc['per_m_wheel',i.split('/')[-1]] =  'mean: {:.3f} std: {:.3f}'.format(round(np.mean(wheel_over_path),3), + round(np.std(wheel_over_path),3))
                df.loc['BAC',i.split('/')[-1]] = 'mean: {:.3f} std: {:.3f}'.format(round(np.mean(bac_manually),3),round(np.std(bac_manually),3))
                df.loc['BAC_new',i.split('/')[-1]] = 'mean: {:.3f} std: {:.3f}'.format(round(np.mean(bac_manually_2),3),round(np.std(bac_manually_2),3))
            if ate:
                if (saving_results):
                    df.loc['final_entropy_norm',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(entropy_bucketed, axis = 0)[-1],3),round(np.std(entropy_bucketed, axis = 0)[-1],3))
                    df.loc['ATE RMSE',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(tot_ATE),3),round(np.std(tot_ATE),3))
                else:
                    df.loc['ATE RMSE',i.split('/')[-1]] =  'mean: {:.3f} std: {:.3f}'.format(round(np.mean(tot_ATE),3),round(np.std(tot_ATE),3))
                    df.loc['final_entropy_norm',i.split('/')[-1]] =  'mean: {:.3f} std: {:.3f}'.format(round(np.mean(entropy_bucketed, axis = 0)[-1],3),round(np.std(entropy_bucketed, axis = 0)[-1],3))
                df.loc['path_len',i.split('/')[-1]] =  'mean: $' + str(np.mean(path_len))[:5] + '$ ' + ' std: $' + str(np.std(path_len))[:5]+'$'
                lc_over_path = list(map(truediv, tot_lc, path_len))
                if (saving_results):
                    df.loc['Loops per m',i.split('/')[-1]] =  'mean: ${:.3f}$ std: ${:.3f}$'.format(round(np.mean(lc_over_path),3),round(np.std(lc_over_path),3))
                else:
                    df.loc['Loops per m',i.split('/')[-1]] =  'mean: {:.3f} std: {:.3f}'.format(round(np.mean(lc_over_path),3),round(np.std(lc_over_path),3))
                
            panda = pd.DataFrame(columns=['time','norm_entropy','area','path_len','exp'])
            cnt_exp = 0
            for tries in range(len(entropy_bucketed)):
                for result in range(len(entropy_bucketed[tries])):
                    panda.loc[cnt_exp,'norm_entropy'] = entropy_bucketed[tries][result]
                    panda.loc[cnt_exp,'time'] = result*2
                    if ('E2' in current):
                        panda.loc[cnt_exp,'area'] = area_bucketed[tries][result]/204.0 #area_bucketed[tries][result]
                    else:
                        panda.loc[cnt_exp,'area'] = area_bucketed[tries][result]/183.0 #area_bucketed[tries][result]
                    panda.loc[cnt_exp,'path_len'] = filtered_path_df[tries][result]
                    cnt_exp += 1
            panda.norm_entropy = panda.norm_entropy.astype(float)
            panda.path_len = panda.path_len.astype(float)
            panda.area = panda.area.astype(float)
            lab_ = i.split('/')[-1].replace('E1_','').replace('E2_','')
            lab_ = "$" + lab_[0:2] + "{"+ lab_[2:] +"}" +"$"
            
            sns.lineplot(data=panda, x='time', ci="sd", y="area", ax=ax_map, color=color[cnt], label=lab_)
            sns.lineplot(data=panda, x='time', ci="sd", y="path_len", ax=ax_map_split, color=color[cnt])

            if ('E2' in current):
                lab = i.split('/')[-1].replace('E2_','')
            else:
                lab = i.split('/')[-1].replace('E1_','')
            lab = "$" + lab[0:2] + "{"+ lab[2:] +"}" +"$"
            print(lab)
            if ('E2' in current):
                atyy = np.array(area_bucketed).astype(np.float32)/204.0
                _n_err = -1.96*np.std(atyy, axis = 0)/np.sqrt(atyy.shape[0]) + np.mean(area_bucketed, axis = 0).astype(np.float32)/204.0
                _p_err = +1.96*np.std(atyy, axis = 0)/np.sqrt(atyy.shape[0]) + np.mean(area_bucketed, axis = 0).astype(np.float32)/204.0
                
                ax_exp.plot(np.mean(filtered_path_df,axis=0), np.mean(area_bucketed, axis = 0)/204.0,color=color[cnt], label=lab)
                ax_exp.fill_between(x=np.mean(filtered_path_df,axis=0).astype(np.float32), y1=_n_err, y2=_p_err ,color=color[cnt], alpha = 0.2)
            else:

                atyy = np.array(area_bucketed).astype(np.float32)/183.0
                _n_err = -1.96*np.std(atyy, axis = 0)/np.sqrt(atyy.shape[0]) + np.mean(area_bucketed, axis = 0).astype(np.float32)/183.0
                _p_err = +1.96*np.std(atyy, axis = 0)/np.sqrt(atyy.shape[0]) + np.mean(area_bucketed, axis = 0).astype(np.float32)/183.0

                ax_exp.plot(np.mean(filtered_path_df,axis=0), np.mean(area_bucketed, axis = 0)/183.0,color=color[cnt], label=lab)
                ax_exp.fill_between(x=np.mean(filtered_path_df,axis=0).astype(np.float32), y1=_n_err, y2=_p_err ,color=color[cnt], alpha = 0.2)
            
            etty = np.array(entropy_bucketed).astype(np.float32)
            _n_err = -1.96*np.std(etty, axis = 0)/np.sqrt(etty.shape[0]) + np.mean(entropy_bucketed, axis = 0).astype(np.float32)
            _p_err = +1.96*np.std(etty, axis = 0)/np.sqrt(etty.shape[0]) + np.mean(entropy_bucketed, axis = 0).astype(np.float32)    
            ax_exp.plot(np.mean(filtered_path_df,axis=0), np.mean(entropy_bucketed, axis=0),color=color[cnt])
            ax_exp.fill_between(x=np.mean(filtered_path_df,axis=0).astype(np.float32), y1=_n_err, y2=_p_err ,color=color[cnt], alpha = 0.2)

            ax_exp.legend(ncol=2)
            
#             ax_ent.legend()

            ax_map.legend(ncol=2)

#             ax_area.legend()

            # plot CDF
#             edges = compute_bin_size(cov_vec)
#             n, bins, patches = ax1.hist(cov_vec, edges, cumulative=True, density=True, histtype='step', label=i.split('/')[-1], color=color[cnt])
#             patches[0].set_xy(patches[0].get_xy()[:-1])
#             #ax1.plot(bins, n,  label=columns[cnt],c=np.random.rand(3,) )
#             if ate:
#                 edges = com',pute_bin_size(pose_cov_vec)
#                 n, bins, patches = ax2.hist(pose_cov_vec, edges, cumulative=True, density=True, histtype='step', label=i.split('/')[-1], color=color[cnt])
#                 patches[0].set_xy(patches[0].get_xy()[:-1])
                #ax2.plot(bins[1:], n,  ,c=np.random.rand(3,) )
            cnt += 1
#             print(tot_area)
            print(len(tot_lc))
            print('---------')
    
#     handles, labels = ax1.get_legend_handles_labels()
#     new_handles = [lines.Line2D([], [], c=h.get_edgecolor()) for h in handles]
#     ax1.legend(handles=new_handles, labels=labels)

#     if ate:
#         handles, labels = ax2.get_legend_handles_labels()
#         new_handles = [lines.Line2D([], [], c=h.get_edgecolor()) for h in handles]
#         ax2.legend(handles=new_handles, labels=labels)
#     fig1.suptitle('E1' if 'E1' in folder else 'E2')
#     fig1.show()
    
#     fig_map.suptitle('E1' if 'E1' in folder else 'E2')
    fig_map.show() 
#     fig_exp.suptitle('E1' if 'E1' in folder else 'E2')
    fig_exp.show()        
    
    separator = "-"
    if (saving_results):
        fig_map.savefig('map-{}-{}-{}.eps'.format('E1' if 'E1' in folder else 'E2',separator.join(columns),separator.join(tests)),format='eps',bbox_inches='tight')
        fig_exp.savefig('exp-{}-{}-{}.eps'.format('E1' if 'E1' in folder else 'E2',separator.join(columns),separator.join(tests)),format='eps',bbox_inches='tight')
    
#     fig_area.suptitle('E1' if 'E1' in folder else 'E2')
#     fig_area.show()        
    
    display(df)
    if (saving_results):
        print(df.to_latex(index=True, escape=False))

In [ ]:
matplotlib.rcdefaults()
matplotlib.rcParams['pdf.fonttype']=42
matplotlib.rcParams['ps.fonttype']=42
font = {
        'weight': 'normal',
        'size': 13,
            }
matplotlib.rc('font', **font) 

columns = ['E1']#,'S_O']#,'S_O','S_DW_O']#['S',]
# tests = ['gridmap_transformation_astar', 'gridmap_transformation_dijkstra', 'costmap_default_astar', 'gridmap_default_astar', 'gridmap_default_astar_scaled']
# tests = ['gridmap_default_dijkstra', 'gridmap_transformation_dijkstra', 'gridmap_transformation_astar']
tests = ["C_I", "C_costmap", "C_gridmap"]
folder = '/media/statordyna/Backup Plus/Test/'
folder_E2 = os.path.join(folder,'E1')

all_BAC = []
all_ATE = []
all_ENT = []
all_PtL = []
all_PmL = []

df_2_ = pd.DataFrame()
# a = get_data(folder_E2, columns, tests, df_2_,0,2, True, True)
a = get_data(folder_E2, columns, tests, df_2_,0,10, True, True)

In [38]:
tab_gcpr = [[], [], []]
map_s = tests
for m in range(len(map_s)):
    print(m)
    for i in range(10):
        tx = ""
        # trial_no = i+1
        # tx+=str(i)+"&"
        tx+=str("{:.3f}".format(round(all_BAC[m][i], 3)))+" & "
        tx+=str("{:.3f}".format(round(all_ATE[m][i], 3)))+" & "
        tx+=str("{:.3f}".format(round(all_ENT[m][i][-1], 3)))+" & "
        tx+=str("{:.3f}".format(round(all_PtL[m][i], 3)))+" & "
        tx+=str("{:.3f}".format(round(all_PmL[m][i], 3)))
        tab_gcpr[m].append(tx)


0
1
2


In [39]:
tab_gcpr = np.array(tab_gcpr)
print(tab_gcpr.shape)
print(map_s)
print("\n")
for i in range(len(tab_gcpr[0])):
    print(str(i)+" & ", end = "")
    print(tab_gcpr[0][i], end=" & ")
    print(tab_gcpr[2][i], end=" & ")
    print(tab_gcpr[1][i], end=" \\\\\n")

print("mean & ", end="")
print(("{:.3f}".format(round(np.mean(all_BAC[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ATE[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ENT[0], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PtL[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PmL[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_BAC[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ATE[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ENT[2], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PtL[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PmL[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_BAC[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ATE[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ENT[1], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PtL[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PmL[1]), 3)))+" \\\\ ") 

print("std & ", end="")
print(("{:.3f}".format(round(np.std(all_BAC[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ATE[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ENT[0], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PtL[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PmL[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_BAC[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ATE[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ENT[2], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PtL[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PmL[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_BAC[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ATE[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ENT[1], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PtL[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PmL[1]), 3)))+" \\\\ ") 

(3, 10)
['C_I', 'C_costmap', 'C_gridmap']


0 & 0.840 & 0.020 & 0.162 & 77.219 & 7.576 & 0.835 & 0.023 & 0.230 & 75.028 & 7.957 & 0.845 & 0.022 & 0.187 & 78.858 & 7.697 \\
1 & 0.817 & 0.030 & 0.174 & 80.955 & 6.621 & 0.845 & 0.080 & 0.175 & 85.204 & 5.892 & 0.844 & 0.038 & 0.136 & 87.517 & 6.399 \\
2 & 0.842 & 0.041 & 0.163 & 79.862 & 6.524 & 0.797 & 0.023 & 0.192 & 80.364 & 6.371 & 0.848 & 0.052 & 0.179 & 85.786 & 7.064 \\
3 & 0.845 & 0.028 & 0.174 & 72.656 & 8.010 & 0.829 & 0.024 & 0.190 & 83.834 & 6.060 & 0.846 & 0.028 & 0.145 & 84.988 & 6.177 \\
4 & 0.823 & 0.031 & 0.181 & 77.213 & 7.395 & 0.845 & 0.036 & 0.202 & 76.426 & 5.980 & 0.825 & 0.071 & 0.131 & 74.325 & 5.032 \\
5 & 0.831 & 0.027 & 0.124 & 76.114 & 6.792 & 0.797 & 0.028 & 0.129 & 52.095 & 2.687 & 0.841 & 0.025 & 0.186 & 83.777 & 6.505 \\
6 & 0.831 & 0.025 & 0.185 & 73.080 & 6.924 & 0.819 & 0.093 & 0.220 & 81.754 & 5.296 & 0.816 & 0.051 & 0.232 & 81.542 & 6.561 \\
7 & 0.828 & 0.032 & 0.183 & 77.270 & 7.390 & 0.826 & 0.030 &

In [ ]:
matplotlib.rcdefaults()
matplotlib.rcParams['pdf.fonttype']=42
matplotlib.rcParams['ps.fonttype']=42
font = {
        'weight': 'normal',
        'size': 13,
            }
matplotlib.rc('font', **font) 

columns = ['E2']#,'S_O']#,'S_O','S_DW_O']#['S',]
# tests = ['gridmap_transformation_astar', 'gridmap_transformation_dijkstra', 'costmap_default_astar', 'gridmap_default_astar', 'gridmap_default_astar_scaled']
# tests = ['gridmap_default_dijkstra', 'gridmap_transformation_dijkstra', 'gridmap_transformation_astar']
tests = ["C_I", "C_costmap", "C_gridmap"]
folder = '/media/statordyna/Backup Plus/Test/'
folder_E2 = os.path.join(folder,'E2')

all_BAC = []
all_ATE = []
all_ENT = []
all_PtL = []
all_PmL = []

df_2_ = pd.DataFrame()
# a = get_data(folder_E2, columns, tests, df_2_,0,2, True, True)
a = get_data(folder_E2, columns, tests, df_2_,0,10, True, True)

In [49]:
tab_gcpr = [[], [], []]
map_s = tests
for m in range(len(map_s)):
    print(m)
    for i in range(10):
        tx = ""
        # trial_no = i+1
        # tx+=str(i)+"&"
        tx+=str("{:.3f}".format(round(all_BAC[m][i], 3)))+" & "
        tx+=str("{:.3f}".format(round(all_ATE[m][i], 3)))+" & "
        tx+=str("{:.3f}".format(round(all_ENT[m][i][-1], 3)))+" & "
        tx+=str("{:.3f}".format(round(all_PtL[m][i], 3)))+" & "
        tx+=str("{:.3f}".format(round(all_PmL[m][i], 3)))
        tab_gcpr[m].append(tx)


0
1
2


In [50]:
tab_gcpr = np.array(tab_gcpr)
print(tab_gcpr.shape)
print(map_s)
print("\n")
for i in range(len(tab_gcpr[0])):
    print(str(i)+" & ", end = "")
    print(tab_gcpr[0][i], end=" & ")
    print(tab_gcpr[2][i], end=" & ")
    print(tab_gcpr[1][i], end=" \\\\\n")

print("mean & ", end="")
print(("{:.3f}".format(round(np.mean(all_BAC[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ATE[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ENT[0], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PtL[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PmL[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_BAC[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ATE[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ENT[2], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PtL[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PmL[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_BAC[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ATE[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_ENT[1], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PtL[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.mean(all_PmL[1]), 3)))+" \\\\ ") 

print("std & ", end="")
print(("{:.3f}".format(round(np.std(all_BAC[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ATE[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ENT[0], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PtL[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PmL[0]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_BAC[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ATE[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ENT[2], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PtL[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PmL[2]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_BAC[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ATE[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_ENT[1], axis=0)[-1], 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PtL[1]), 3)))+" & ", end="")
print(("{:.3f}".format(round(np.std(all_PmL[1]), 3)))+" \\\\ ") 

(3, 10)
['C_I', 'C_costmap', 'C_gridmap']


0 & 0.781 & 0.020 & 0.080 & 84.700 & 6.765 & 0.817 & 0.033 & 0.252 & 86.457 & 6.084 & 0.772 & 0.055 & 0.160 & 79.041 & 4.884 \\
1 & 0.779 & 0.026 & 0.177 & 72.734 & 5.692 & 0.820 & 0.109 & 0.220 & 90.004 & 4.989 & 0.778 & 0.027 & 0.160 & 89.807 & 5.723 \\
2 & 0.811 & 0.075 & 0.262 & 77.507 & 5.987 & 0.744 & 0.050 & 0.199 & 84.777 & 4.966 & 0.805 & 0.037 & 0.164 & 92.816 & 6.281 \\
3 & 0.833 & 0.020 & 0.084 & 86.342 & 6.440 & 0.740 & 0.062 & 0.155 & 89.814 & 5.656 & 0.800 & 0.080 & 0.244 & 83.942 & 5.492 \\
4 & 0.808 & 0.022 & 0.133 & 83.184 & 6.323 & 0.771 & 0.332 & 0.245 & 87.955 & 4.821 & 0.763 & 0.022 & 0.256 & 86.696 & 6.875 \\
5 & 0.807 & 0.043 & 0.195 & 72.289 & 6.778 & 0.770 & 0.153 & 0.215 & 89.608 & 4.308 & 0.781 & 0.023 & 0.172 & 86.625 & 6.280 \\
6 & 0.772 & 0.022 & 0.193 & 81.013 & 6.505 & 0.816 & 0.021 & 0.166 & 92.556 & 6.266 & 0.781 & 0.017 & 0.191 & 87.173 & 6.711 \\
7 & 0.805 & 0.048 & 0.244 & 81.786 & 6.016 & 0.809 & 0.037 &